In [1]:
from sklearn.metrics import roc_auc_score
import logging
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight
from keras.callbacks import *
import gensim
import tensorflow as tf
import pandas as pd
from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn import preprocessing

In [4]:
MODEL_PATH = "../models/"

In [5]:
model_name = 'bidirectional_lstm_ensemble_per_class'

In [6]:
DATA_PATH = "../data/"
data = pd.read_csv(DATA_PATH + "preprocessed/train_heavy_clean_no-stopwords.csv")

In [7]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [8]:
data.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,heavy_clean
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,' aww ! matches background colour seemingly st...
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"hey man , really trying edit war . guy constan..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,""" cannotmake real suggestions improvement - wo..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,", sir , hero . chance remember page ?"


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data[['comment_text', 'heavy_clean']], data[list_classes], test_size=0.33, random_state=42)

In [10]:
tokenizer = Tokenizer(num_words=None, oov_token=0)
tokenizer.fit_on_texts(X_train['comment_text'])
word_index = tokenizer.word_index

In [11]:
# X_train.loc[pd.isnull(X_train['heavy_clean'])]['heavy_clean'] = X_train[pd.isnull(X_train['heavy_clean'])]['comment_text'].apply(heavy_clean)

# X_train.loc[8846]['heavy_clean'] = heavy_clean(X_train.loc[8846]['comment_text'])

# X_train[pd.isnull(X_train['heavy_clean'])]

In [12]:
embeddings_index_w2v = gensim.models.KeyedVectors.load_word2vec_format('../../../embeddings/GoogleNews-vectors-negative300.bin', binary = True)

In [13]:
max_length = 200

In [14]:
print('Found %s word vectors.' % len(embeddings_index_w2v.vocab))
embedding_matrix = np.zeros((len(word_index) + 2, 300))
oov = []
embedding_matrix[0] = np.random.rand(1, 300)
embedding_matrix[-1] = np.zeros((1, 300))
for word, i in word_index.items():
    word = str(word)
    if word in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower().capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower().capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    else:
        oov.append(word)
embedding_size = 300

Found 3000000 word vectors.


In [15]:
len(word_index)

164436

In [16]:
len(set(word_index) & set(embeddings_index_w2v.vocab))

58515

In [17]:
len(oov)

80765

In [18]:
embedding_matrix

array([[ 0.13909956,  0.49121553,  0.13078724, ...,  0.30649366,
         0.39451936,  0.80310854],
       [ 0.07468531,  0.09791063,  0.04645062, ...,  0.00341549,
         0.04440133, -0.06421115],
       [ 0.01803273,  0.0796839 ,  0.03568782, ...,  0.03077838,
        -0.01652214, -0.09403456],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04803049, -0.03817808,  0.03217427, ...,  0.00434891,
        -0.0180884 ,  0.08744012],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length, trainable = False)

In [20]:
adam = keras.optimizers.Adam(clipnorm=1.)

In [21]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [22]:
x_train_texts = tokenizer.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_test_texts = tokenizer.texts_to_sequences(X_test['comment_text'])
x_test_texts = pad_sequences(x_test_texts, maxlen=max_length, padding='post')

In [23]:
batch_size = 32
epochs = 3

In [24]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train.values[:, 0]), y_train.values[:, 0])
class_weights = {i:x for i, x in enumerate(list(class_weights))}
class_weights

{0: 0.5527967652868119, 1: 5.235138576045441}

In [25]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(embedded_sequences)
lstm2 = Bidirectional(CuDNNLSTM(5))(lstm1)
output = Dense(units=1, activation='sigmoid')(lstm2)
model_1 = Model(sequence_input, output)
model_1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_1.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_1.fit(x_train_texts, y_train.values[:, 0], validation_data=(x_test_texts, y_test.values[:, 0]),
              epochs=epochs, batch_size=batch_size, callbacks=[RocAuc], class_weight = class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 40)           51520     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10)                1880      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 49,384,811
Trainable params: 53,411
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/3
106912/106912 [==============================] - 234s 2ms/step -

In [26]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train.values[:, 1]), y_train.values[:, 1])
class_weights = {i:x for i, x in enumerate(list(class_weights))}
class_weights

{0: 0.505049932447115, 1: 50.005612722170255}

In [27]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(embedded_sequences)
lstm2 = Bidirectional(CuDNNLSTM(5))(lstm1)
output = Dense(units=1, activation='sigmoid')(lstm2)
model_2 = Model(sequence_input, output)
model_2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_2.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_2.fit(x_train_texts, y_train.values[:, 1], validation_data=(x_test_texts, y_test.values[:, 1]),
              epochs=epochs, batch_size=batch_size, callbacks=[RocAuc], class_weight = class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200, 40)           51520     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 10)                1880      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 49,384,811
Trainable params: 53,411
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/3
106912/106912 [==============================] - 234s 2ms/step -

In [28]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train.values[:, 2]), y_train.values[:, 2])
class_weights = {i:x for i, x in enumerate(list(class_weights))}
class_weights

{0: 0.5277311588050625, 1: 9.515129939480243}

In [29]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(embedded_sequences)
lstm2 = Bidirectional(CuDNNLSTM(5))(lstm1)
output = Dense(units=1, activation='sigmoid')(lstm2)
model_3 = Model(sequence_input, output)
model_3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_3.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_3.fit(x_train_texts, y_train.values[:, 2], validation_data=(x_test_texts, y_test.values[:, 2]),
              epochs=epochs, batch_size=batch_size, callbacks=[RocAuc], class_weight = class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200, 40)           51520     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 10)                1880      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 49,384,811
Trainable params: 53,411
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/3
106912/106912 [==============================] - 234s 2ms/step -

In [30]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train.values[:, 3]), y_train.values[:, 3])
class_weights = {i:x for i, x in enumerate(list(class_weights))}
class_weights

{0: 0.5015292815191489, 1: 163.9754601226994}

In [32]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(embedded_sequences)
lstm2 = Bidirectional(CuDNNLSTM(5))(lstm1)
output = Dense(units=1, activation='sigmoid')(lstm2)
model_4 = Model(sequence_input, output)
model_4.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_4.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_4.fit(x_train_texts, y_train.values[:, 3], validation_data=(x_test_texts, y_test.values[:, 3]),
              epochs=epochs, batch_size=batch_size, callbacks=[RocAuc], class_weight = class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 200, 40)           51520     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 10)                1880      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 49,384,811
Trainable params: 53,411
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/3
106912/106912 [==============================] - 231s 2ms/step -

In [33]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train.values[:, 4]), y_train.values[:, 4])
class_weights = {i:x for i, x in enumerate(list(class_weights))}
class_weights

{0: 0.5257381144397018, 1: 10.213221245701185}

In [34]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(embedded_sequences)
lstm2 = Bidirectional(CuDNNLSTM(5))(lstm1)
output = Dense(units=1, activation='sigmoid')(lstm2)
model_5 = Model(sequence_input, output)
model_5.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_5.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_5.fit(x_train_texts, y_train.values[:, 4], validation_data=(x_test_texts, y_test.values[:, 4]),
              epochs=epochs, batch_size=batch_size, callbacks=[RocAuc], class_weight = class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 200, 40)           51520     
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 10)                1880      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 49,384,811
Trainable params: 53,411
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/3
106912/106912 [==============================] - 233s 2ms/step -

In [35]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train.values[:, 5]), y_train.values[:, 5])
class_weights = {i:x for i, x in enumerate(list(class_weights))}
class_weights

{0: 0.5044065749495178, 1: 57.23340471092077}

In [36]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(embedded_sequences)
lstm2 = Bidirectional(CuDNNLSTM(5))(lstm1)
output = Dense(units=1, activation='sigmoid')(lstm2)
model_6 = Model(sequence_input, output)
model_6.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_6.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_6.fit(x_train_texts, y_train.values[:, 5], validation_data=(x_test_texts, y_test.values[:, 5]),
              epochs=epochs, batch_size=batch_size, callbacks=[RocAuc], class_weight = class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 200, 40)           51520     
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 10)                1880      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 49,384,811
Trainable params: 53,411
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/3
106912/106912 [==============================] - 235s 2ms/step -

In [37]:
model_json = model_1.to_json()
with open("../models/bidirectional_lstm_ensemble_per_class_model_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_1.save_weights("../models/bidirectional_lstm_ensemble_per_class_model_1.h5")
print("Saved model to disk")
 

model_json = model_2.to_json()
with open("../models/bidirectional_lstm_ensemble_per_class_model_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_2.save_weights("../models/bidirectional_lstm_ensemble_per_class_model_2.h5")
print("Saved model to disk")
 

model_json = model_3.to_json()
with open("../models/bidirectional_lstm_ensemble_per_class_model_3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_3.save_weights("../models/bidirectional_lstm_ensemble_per_class_model_3.h5")
print("Saved model to disk")
 

model_json = model_4.to_json()
with open("../models/bidirectional_lstm_ensemble_per_class_model_4.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_4.save_weights("../models/bidirectional_lstm_ensemble_per_class_model_4.h5")
print("Saved model to disk")
 

model_json = model_5.to_json()
with open("../models/bidirectional_lstm_ensemble_per_class_model_5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_5.save_weights("../models/bidirectional_lstm_ensemble_per_class_model_5.h5")
print("Saved model to disk")

model_json = model_6.to_json()
with open("../models/bidirectional_lstm_ensemble_per_class_model_6.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_6.save_weights("../models/bidirectional_lstm_ensemble_per_class_model_6.h5")
print("Saved model to disk")
 

Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk


In [39]:
# combine to predict
combined_sequence_input = Input(shape=(max_length,), dtype='int32')
combined_embedded_sequences = embedding_layer(combined_sequence_input)
combined_conc = concatenate([model_1(combined_embedded_sequences),
             model_2(combined_embedded_sequences),
             model_3(combined_embedded_sequences),
             model_4(combined_embedded_sequences),
             model_5(combined_embedded_sequences),
             model_6(combined_embedded_sequences),])
combined_model = Model(combined_sequence_input, combined_conc)
combined_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])    



ValueError: Shape must be rank 3 but is rank 2 for 'model_1_1/bidirectional_1/Tile' (op: 'Tile') with input shapes: [?,300,1], [2].

In [ ]:
y_pred = combined_model.predict(x_test, batch_size=4, verbose=1)
submission = pd.read_csv('../submissions/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('bd_lstm_per_class1.csv', index=False)

# Error analysis

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
# class_names = [0, 1]
# cnf_matrix = confusion_matrix(y_test.values[:, 3], np.round(model_pred))
# np.set_printoptions(precision=2)

# # Plot non-normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=class_names,
#                       title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
#                       title='Normalized confusion matrix')

# plt.show()

In [ ]:
# list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# with open("../ling_src/obscene_words.txt", "r") as f:
#     content = f.readlines()
# swear_words = set([x.strip() for x in content])
# print('Number of swear words in vocabulary: ', len(swear_words))

# manual_pred_threat = []
# for text in data['comment_text'].values:
#     if any(word in text.lower() for word in swear_words) and 'will' in text.lower():
#         manual_pred_threat.append(1)
#     else:
#         manual_pred_threat.append(0)

# roc_auc_score(data['threat'].values, manual_pred_threat)

# class_names = [0, 1]
# cnf_matrix = confusion_matrix(data['threat'].values, manual_pred_threat)
# np.set_printoptions(precision=2)

# # Plot non-normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=class_names,
#                       title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
#                       title='Normalized confusion matrix')

# plt.show()